##                                                        Regression Tree

Decision tree is non parametric Machine learning algorithm that can be used both for regression and classification. The idea behind it is to inductively split the feature space into different subspaces during the training (commonly referred as building the tree). Each new point will be classified/regressed(?) according to the subspace it belongs to. Of course the important thing of a decision tree is to choose a procedure to carry out the splittings. All these procedures are based on the idea of maximization or minimization of some quantity; different objects are minimized for regression and classification. A decision tree is built using a top down, greedy approach; where top down means that initially every data point belongs to the same subspace (which at this point is the whole space) and they are gradually end up into different subuspaces. And greedy means that at each step the best possible split is made (thus we are deciding locally the best course of action).  
Let's start with regression.  
To fix some notation let $$X=(x_i^j)_{i=1,\dots, n\\ j=1,\dots,m}$$ be our training data and let $y=(y_1,\dots,y_n)$ be the associated target values. As always our task is to associate the 'right' value to a point which is not in our training set, that is to build a function $\tilde{f}:\mathbb{R}^m\rightarrow\mathbb{R}$ which will predict the correct value of a data point $x$.  
When we build a tree as a result we obtain a set of rectangles $\{R_1,\dots,R_K\}$ and a function $\tilde{f}:\mathbb{R}^m\rightarrow\mathbb{R}$ such that   

$$\cup_{i=1}^K R_i=\mathbb{R}^m,$$
$$R_i\cap R_j=\varnothing,\ \ \  \forall i\neq j, $$
$$f_{\vert R_i}(x)=\hat{y}_i,$$
where 
$$\hat{y}_i=\frac{\sum_{j=1}^n y_j\mathbb{1}_{R_i}(x_j)}{\sum_{j=1}^n\mathbb{1}_{R_i}(x_j)}.$$
What does this mean? It means that we have built a partition of the feature space $\mathbb{R}^m$ made of a certain number of boxes and defined a function $\tilde{f}$ which takes the constant value $\hat{y}_i$ on the $i-$th box. The value $\hat{y}_i$ is the avarage of the of the values of the train targets which corresponding data point is contained in $R_i$. 
To find the subspaces we would like to minimize the following quanitity (sum of the squares of the errors)
$$\sum_{j\in J}\sum_{i=1}^n(y_i-\hat{y}_i)^2\chi_{R_j}(x_i)$$
among all the partitions of $\mathbb{R}^m$ into rectangles $R_j$. The choice of rectangles is made for simplicity and to mantain interpretability. This quantity depends on the partition choosen and thus to find its minimum we would have to look into the space of all possible partitions of $\mathbb{R}^m$ into rectangles. This approach is not feasible in practice because this space is huge. We will take a greedy approach. Let's go through the first step.  
To perform the first splitting we take into accout couples of the form $(j,s)$ where $j\in\{1,\dots,m\}$ and $s\in\mathbb{R}$ is the threshold we will use to divide the space and we define the two families of rectangles
$$R_1(j,s):=\{x\in\mathbb{R}^m, x_j<s\},\ \ \ \ \ \ \ \ \ R_2(j,s):=\{x\in\mathbb{R}^m, x_j\geq s\}.$$
We now look for the couple $(j,s)$ that minimizes the error
$$\sum_{i=1}^m(y_i-\hat{y}_i)^2\chi_{R_1(j,s)}(x_i)+\sum_{i=1}^m(y_i-\hat{y}_i)^2\chi_{R_2(j,s)}(x_i).$$

At the end of this first step we have a partition of $\mathbb{R}^m$ in two rectangles and we are ready to apply the same procedure on them. At each step we make an optimal choice and as always happens with greedy algorithms the combination of these locally optimal choices do not lead to a globally optimal choice. We observe that if the feature space is discrete the task of finding $s$ is trivial (because if your features can take only values $-1$, $1$, then any choice of $-1<s<1$ is equally good as a threshold). In the continuous case $s$ can be found using the module fminbound() in scipy.optimization.  

## Classification Tree

In this case we have $Y=\{y_1,\dots,y_l\}$ classes and we want to find a function $f:\mathbb{R}^m\rightarrow Y$. As before we will end up with a partition of the domain in rectangles. The rectangles will be defined iteratively following a greedy approach, that is at each step we will minimize a quantity. The quantity most widely used are the Gini index and the entropy (we can also use the misclassification rate). 
For simplicity let's assume we have only two classes $y_1,\ y_2$ and let $g$ be the function that associates any element of the training set to the correct label.  

For the task of binary classification, as above we consider the rectangles
$$R_1(j,s):=\{x\in\mathbb{R}^m, x_j<s\},\ \ \ \ \ \ \ \ \ R_2(j,s):=\{x\in\mathbb{R}^m, x_j\geq s\}.$$
We compute the quantities 
$$p_{ik}=\frac{\big\vert R_i(j,s) \cap g^{-1}(y_k)\big\vert}{\big\vert g^{-1}(y_k)\big\vert},$$
that is the number of elements in the rectangle $i$ that belong to the class $k$ divided by the number of elements that belong to the class $k$ (if we have already performed some splittings these quantities are computed taking into account the data points inside a rectangle and not in $\mathbb{R}^m$).  
The Gini index of the rectangle is computed as 
$$\text{Gini}(R_i(j,s))=1-p_{i1}^2+p_{i2}^2$$
The Gini index of the split is computed as 
$$\text{Gini}(j,s)=\frac{\big\vert \Sigma_{i=1}^n \mathbb{1}_{R_1(j,s)}(x_i)\big\vert}{\big\vert \Sigma_{i=1}^n \mathbb{1}_{R_1(j,s)\cup R_2(j,s)}(x_i)\big\vert}\text{Gini}(R_1(j,s))+\frac{\big\vert \Sigma_{i=1}^n \mathbb{1}_{R_2(j,s)}(x_i)\big\vert}{\big\vert \Sigma_{i=1}^n \mathbb{1}_{R_1(j,s)\cup R_2(j,s)}(x_i)\big\vert}\text{Gini}(R_2(j,s)).$$
This is the quantity to be minimized at every step.

Entropy

## Random Forest

We have seen how to grow a single tree. The problem with trees is that they suffer from high variance, that is if we randomly select a subset of the given dataset and grow the tree on it, we will likely end up with a different tree (different partition and different function). In a sense, the final function depends "too much" on the choosen training data. To reduce this dependence Breiman introduced the concept of random forest.  
Random forests are called this way because they rely on two random choices.  
* The first is to use bagging. Bagging is a technique used to reduce variance by extracting (with replacement) a certain number of subsamples from the original dataset and training one different model per subsample (each subsample has the same number of elements of the original model). If the goal is regression the final model will be the avarage of the different models, while for classification the output class is choosen by majority vote (other approaches are possible for both cases). We expect bagging to work because we assume that the subsample is representative of the entire dataset, since the training examples have the same underlining distribution. Being the exactions random, it it possible that some of the examples will be used more than once, while others will not be used. This feature turns out to be useful, indeed, it can be proved that on avarage every tree uses only two thirds of the whole dataset. The datapoints not used to grow the three are called out-of-bag observations and can be used to estimate the error.  
* The second is to grow each tree on a random subset of the features. The purpouse of this is to reduce the correlation among the trees. Indeed if one of the features is too discriminative with respect to the others, all the trees will put it at the first node, and all the bagged trees will look quite similar. The number of features influences the random forest error rate and it can be choosen using out-of-bag error.  

When using random forests the trees are grown to full extent and no pruning is used. Moreover random forests does not overfit and runs efficiently on large datasets
